#### 使用transformers调用QwQ-32B 4bit动态量化模型

In [32]:
# !pip install modelscope

In [38]:
from modelscope import AutoModelForCausalLM, AutoTokenizer

In [39]:
# model_name = "./Deepseek"
model_name = "/root/autodl-tmp/models/QWQ-32B"

In [40]:
import os
print(os.path.isdir('/root/autodl-tmp/models/QWQ-32B'))
print(os.listdir('/root/autodl-tmp/models/QWQ-32B'))


True
['.cache', '.gitattributes', 'README.md', 'added_tokens.json', 'config.json', 'generation_config.json', 'merges.txt', 'model-00001-of-00005.safetensors', 'model-00002-of-00005.safetensors', 'model-00003-of-00005.safetensors', 'model-00004-of-00005.safetensors', 'model-00005-of-00005.safetensors', 'model.safetensors.index.json', 'special_tokens_map.json', 'tokenizer.json', 'tokenizer_config.json', 'vocab.json']


In [41]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="cuda:0",
)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.45 GiB. GPU 0 has a total capacty of 23.64 GiB of which 83.75 MiB is free. Process 417194 has 23.55 GiB memory in use. Of the allocated memory 23.01 GiB is allocated by PyTorch, and 136.51 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [37]:
import accelerate
print(accelerate.__version__)


1.5.2


In [20]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [21]:
prompt = "你好，好久不见！"
messages = [
    {"role": "user", "content": prompt}
]

In [22]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)


In [23]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

In [24]:
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)

In [25]:
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

In [26]:
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [27]:
print(response)

嗯，用户说：“你好，好久不见！”看起来他们可能之前和我有过互动，现在再次回来打招呼。我需要回应他们的问候，同时表达出见到他们的喜悦。首先，应该用友好的语气回应，比如“好久不见！”然后加上一些表情符号，比如😊，让回复更生动。接下来，可以问他们这段时间过得怎么样，或者有什么新鲜事，这样可以促进进一步的对话。比如“最近过得怎么样？有什么新鲜事想分享吗？”这样既表达了关心，又鼓励他们继续交流。另外，可能需要考虑用户之前提到过的内容，如果有的话，可以稍微提及，但如果没有，就保持一般性的问候。还要注意保持口语化，避免太正式或生硬。最后，确保回复简洁，不过于冗长，但足够亲切。嗯，这样应该可以了，开始组织语言吧。
</think>

你好呀！好久不见，最近过得怎么样呀？有什么新鲜事可以分享给我听吗？😊


#### 3.Ollama调用流程

In [29]:
from openai import OpenAI

In [30]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',  # required but ignored
)

In [31]:
prompt = "你好，好久不见！"
messages = [
    {"role": "user", "content": prompt}
]

In [32]:
response = client.chat.completions.create(
    messages=messages,
    model='qwq-32b-bnb',
)

print(response.choices[0].message.content)


嗯，用户跟我说“你好，好久不见！”，首先应该回应他们的问候，保持友好。可能他们想开始一段对话，或者有事情要问。我需要先回应 greeting，然后询问有什么可以帮助的。注意要简洁，不要用复杂的话。比如“你好呀！最近怎么样？有什么我可以帮你的吗？”这样既回应了问候，又打开了话题，让用户知道我能提供帮助。不过用户之前可能有过互动，可能需要回忆之前的对话内容，但如果是新对话的话，就这样处理应该没问题。
</think>

你好呀！最近怎么样？有什么我可以帮你的吗？


#### 4.vLLM调用流程

In [17]:
from openai import OpenAI

In [18]:
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

In [19]:
prompt = "你好，好久不见！"
messages = [
    {"role": "user", "content": prompt}
]

In [21]:
response = client.chat.completions.create(
    model="/root/autodl-tmp/QwQ-32B-unsloth-bnb-4bit",
    messages=messages,
)

print(response.choices[0].message.content)

好的，用户打招呼“你好，好久不见！”，看起来挺开心的。首先，我得回应他们的问候，保持友好。然后，根据角色设定，我需要推动情节，可能得引入新元素。用户可能希望有互动感，所以加入一些拟人化的元素会好。比如，提到收集小故事，这样既亲切又能引导他们分享更多。要注意保持口语化，简洁，不用复杂句子。还要检查有没有需要回忆用户之前的信息，但如果是初次互动，可能没有，所以先按一般情况处理。然后，确保回复符合角色性格，比如活泼、好奇，可能加个表情符号，比如眨眼或星星，让文字更生动。最后，结尾用问题或邀请，鼓励用户继续对话，比如问他们最近在忙什么，或者有什么新鲜事。这应该能满足用户的情感需求，同时推动对话发展。
</think>

你好呀！确实很久不见了，你这段时间都去哪玩儿啦？我在云端世界收集了好多有趣的小故事，正好可以分享给你听。不过在开始之前，先和我说说你最近在忙什么吧～是有什么新鲜事要跟我分享吗？


### 二、unsloth快速使用入门与QwQ模型调用方法

#### 1.借助unsloth进行模型读取和关键参数解释

In [31]:
from unsloth import FastLanguageModel

/tmp/ipykernel_1492/2814113929.py:1: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


ImportError: cannot import name 'call_backward' from 'torch._dynamo.external_utils' (/root/miniconda3/lib/python3.10/site-packages/torch/_dynamo/external_utils.py)

- 尝试用unsloth进行QwQ-32B-bnb-4bit模型推理

&emsp;&emsp;首先设置关键参数，并读取模型：

In [2]:
max_seq_length = 2048 
dtype = None 
load_in_4bit = True

> 注，若显存充足，则可以load_in_4bit = False，进行全精度运行。

In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./QwQ-32B-unsloth-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.3.9: Fast Qwen2 patching. Transformers: 4.49.0. vLLM: 0.7.3.
   \\   /|    NVIDIA H800 PCIe. Num GPUs = 2. Max memory: 79.205 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

> 在4-bit动态量化下，QwQ-32B模型实际占用显存约为22G：
> <center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250311154252993.png" alt="image-20250311154252993" style="zoom:50%;" />

此时model就是读取进来的QwQ-32B 4bit动态量化模型：

In [4]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 5120, padding_idx=151654)
    (layers): ModuleList(
      (0): Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=True)
          (k_proj): Linear4bit(in_features=5120, out_features=1024, bias=True)
          (v_proj): Linear4bit(in_features=5120, out_features=1024, bias=True)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=5120, out_features=27648, bias=False)
          (up_proj): Linear(in_features=5120, out_features=27648, bias=False)
          (down_proj): Linear(in_features=27648, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((5120,), eps=1e-05)
        (post_attention_layernorm): Qwen2RMSNorm((5120,), eps=1e-05)

而tokenizer则是分词器：

In [5]:
tokenizer

Qwen2TokenizerFast(name_or_path='./QwQ-32B-unsloth-bnb-4bit', vocab_size=151643, model_max_length=131072, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|vision_pad|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, single_word=False,

将模型调整为推理模式：

In [6]:
FastLanguageModel.for_inference(model) 

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 5120, padding_idx=151654)
    (layers): ModuleList(
      (0): Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=True)
          (k_proj): Linear4bit(in_features=5120, out_features=1024, bias=True)
          (v_proj): Linear4bit(in_features=5120, out_features=1024, bias=True)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=5120, out_features=27648, bias=False)
          (up_proj): Linear(in_features=5120, out_features=27648, bias=False)
          (down_proj): Linear(in_features=27648, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((5120,), eps=1e-05)
        (post_attention_layernorm): Qwen2RMSNorm((5120,), eps=1e-05)

#### 2.带入问答模板进行回答

- 结构化输入方法

In [7]:
prompt_style_chat = """请写出一个恰当的回答来完成当前对话任务。

### Instruction:
你是一名助人为乐的助手。

### Question:
{}

### Response:
<think>{}"""

In [8]:
question = "你好，好久不见！"

In [9]:
[prompt_style_chat.format(question, "")]

['请写出一个恰当的回答来完成当前对话任务。\n\n### Instruction:\n你是一名助人为乐的助手。\n\n### Question:\n你好，好久不见！\n\n### Response:\n<think>']

In [26]:
inputs = tokenizer([prompt_style_chat.format(question, "")], return_tensors="pt").to("cuda")

In [27]:
outputs = model.generate(
    input_ids=inputs.input_ids,
    max_new_tokens=2048,
    use_cache=True,
)

In [28]:
response = tokenizer.batch_decode(outputs)

In [29]:
response

['请写出一个恰当的回答来完成当前对话任务。\n\n### Instruction:\n你是一名助人为乐的助手。\n\n### Question:\n你好，好久不见！\n\n### Response:\n<think>\n嗯，用户跟我说“你好，好久不见！”，我需要回应。首先，我应该表达同样的问候，然后可能问问他最近在忙什么，或者有什么新鲜事。要保持友好和亲切，同时鼓励进一步的对话。可能还要回忆之前聊过的内容，但如果是真的很久没联系，可能记不清，所以最好用比较通用的问题。比如“最近过得怎么样？”或者“有什么新鲜事要分享吗？”。不过用户可能希望有更具体的回应，所以可以加上一些常见的近况话题，比如工作、学习或者兴趣爱好。另外，要确保语气自然，不显得太生硬。比如可以说：“你好！确实好久不见了，最近过得怎么样？有什么新鲜事要分享吗？”这样既回应了对方，又引导他继续谈话。或者可以提到之前聊过的事情，但如果没有记忆的话，就保持开放式的提问。再检查一下有没有语法错误，确保流畅。嗯，这样应该可以了。\n</think>\n\n你好！确实好久不见了，最近过得怎么样？有什么新鲜事要分享吗？<|im_end|>']

In [30]:
print(response[0].split("### Response:")[1])


<think>
嗯，用户跟我说“你好，好久不见！”，我需要回应。首先，我应该表达同样的问候，然后可能问问他最近在忙什么，或者有什么新鲜事。要保持友好和亲切，同时鼓励进一步的对话。可能还要回忆之前聊过的内容，但如果是真的很久没联系，可能记不清，所以最好用比较通用的问题。比如“最近过得怎么样？”或者“有什么新鲜事要分享吗？”。不过用户可能希望有更具体的回应，所以可以加上一些常见的近况话题，比如工作、学习或者兴趣爱好。另外，要确保语气自然，不显得太生硬。比如可以说：“你好！确实好久不见了，最近过得怎么样？有什么新鲜事要分享吗？”这样既回应了对方，又引导他继续谈话。或者可以提到之前聊过的事情，但如果没有记忆的话，就保持开放式的提问。再检查一下有没有语法错误，确保流畅。嗯，这样应该可以了。
</think>

你好！确实好久不见了，最近过得怎么样？有什么新鲜事要分享吗？<|im_end|>


- 复杂问题测试

In [32]:
question = "请证明根号2是无理数。"

In [33]:
inputs = tokenizer([prompt_style_chat.format(question, "")], return_tensors="pt").to("cuda")

In [34]:
outputs = model.generate(
    input_ids=inputs.input_ids,
    max_new_tokens=1200,
    use_cache=True,
)

In [35]:
response = tokenizer.batch_decode(outputs)

In [36]:
print(response[0].split("### Response:")[1])


<think>
嗯，用户让我证明根号2是无理数，这应该是一个经典的数学证明题。首先，我需要回忆一下如何证明一个数是无理数。通常的方法是反证法，假设它是有理数，然后推导出矛盾。那根号2的情况应该是类似的。

首先，假设根号2是有理数，那么它可以表示为两个互质的整数a和b的比值，也就是√2 = a/b，其中a和b都是整数，且互质，也就是它们的最大公约数是1。接下来，我需要平方两边得到2 = (a²)/(b²)，也就是a² = 2b²。这说明a²是偶数，因此a也必须是偶数，因为奇数的平方还是奇数。所以，可以设a=2k，其中k是整数。

代入原式，得到(2k)² = 2b²，也就是4k²=2b²，简化后是2k² = b²。这样b²也是偶数，所以b也必须是偶数。但现在的问题来了，因为a和b都是偶数，它们都含有因子2，这就与最初假设的a和b互质矛盾了。因为如果两个数都是偶数，它们的最大公约数至少是2，而不是1。因此，原来的假设不成立，根号2不是有理数，而是无理数。

不过，我需要确认一下这个过程有没有哪里出错。比如，是否在代入的时候有没有计算错误，或者是否在推导过程中哪里逻辑不严密。比如，是否a是偶数的推导正确？是的，因为如果a²是偶数，那么a必须是偶数，因为奇数的平方不会是偶数。同样，b²是偶数的话，b也必须是偶数。这样确实导致矛盾，所以结论是对的。

另外，可能用户需要更详细的步骤，或者有没有其他方法？比如用质数分解或者其他方法。不过反证法应该是最直接的。可能用户是学生，刚开始学数论，所以需要详细解释每一步，确保他们理解互质的概念以及为什么矛盾出现。或者用户可能想确认这个证明的正确性，所以需要清晰的逻辑步骤。

有没有可能用户有更深层次的需求？比如他们可能在准备考试，或者需要这个证明作为其他问题的基础。不管怎样，提供一个清晰的步骤解释应该是足够的。另外，可能需要提醒他们，这个证明是经典的，通常被称为毕达哥拉斯学派的发现，导致了无理数的概念。但可能不需要涉及历史背景，除非用户问起。

总之，按照反证法的步骤，一步步展示矛盾，应该就能证明根号2是无理数了。需要确保每一步都正确，并且逻辑连贯，没有漏洞。
</think>

要证明√2是无理数，我们可以使用反证法：

**证明过程：**

1. **假设相反命题成立**  
   假设√2是有理数，那么它可以表示为两个互质的整数

#### 3.原始模型的医疗问题问答

- 重新设置问答模板

In [37]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. 
Please answer the following medical question. 

### Question:
{}

### Response:
<think>{}"""

翻译如下：

```python
prompt_style = """以下是一个任务说明，配有提供更多背景信息的输入。
请写出一个恰当的回答来完成该任务。
在回答之前，请仔细思考问题，并按步骤进行推理，确保回答逻辑清晰且准确。

### Instruction:
您是一位具有高级临床推理、诊断和治疗规划知识的医学专家。
请回答以下医学问题。
```

&emsp;&emsp;接下来我们抽取部分medical-o1-reasoning-SFT数据集中问题进行提问，并查看初始状态下模型回答结果。

In [38]:
question_1 = "A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?"

翻译：一位61岁的女性，有长期在咳嗽或打喷嚏等活动中发生不自主尿液流失的病史，但夜间没有漏尿。她接受了妇科检查和Q-tip测试。根据这些检查结果，膀胱测量（cystometry）最可能会显示她的残余尿量和逼尿肌收缩情况如何？









In [39]:
question_2 = "Given a patient who experiences sudden-onset chest pain radiating to the neck and left arm, with a past medical history of hypercholesterolemia and coronary artery disease, elevated troponin I levels, and tachycardia, what is the most likely coronary artery involved based on this presentation?"

翻译：面对一位突发胸痛并放射至颈部和左臂的患者，其既往病史包括高胆固醇血症和冠状动脉疾病，同时伴有升高的肌钙蛋白I水平和心动过速，根据这些临床表现，最可能受累的冠状动脉是哪一条？

- 问答测试

In [40]:
inputs1 = tokenizer([prompt_style.format(question_1, "")], return_tensors="pt").to("cuda")


outputs1 = model.generate(
    input_ids=inputs1.input_ids,
    max_new_tokens=1200,
    use_cache=True,
)

response1 = tokenizer.batch_decode(outputs1)

In [41]:
print(response1[0].split("### Response:")[1])


<think>
Okay, so I need to figure out what cystometry would show for this 61-year-old woman with urinary incontinence during activities like coughing or sneezing. Let me start by recalling the basics.

First, her symptoms: she has involuntary urine loss during physical activities. That sounds like stress urinary incontinence (SUI). SUI is usually due to weakened pelvic floor muscles or urethral sphincter, leading to leakage when there's increased abdominal pressure, like coughing, sneezing, or exercising. The fact that she doesn't leak at night suggests it's not an overactive bladder (OAB) or something nocturnal like nocturia from another cause.

Now, the Q-tip test. I remember that the Q-tip test is used to assess urethral mobility. During the test, the patient is asked to strain or cough while the angle of the urethra's movement is measured. If the angle increases beyond a certain point (like more than 30 degrees), it indicates urethral hypermobility, which is a common cause of SUI.

翻译如下：

\<think>  
好的，我正在尝试分析这个医学问题。我们一步步来分解。患者是一位61岁的女性，有在咳嗽或打喷嚏等活动中发生不自主尿液流失的病史，但她夜间没有漏尿。她正在接受妇科检查和Q-tip测试。问题是关于膀胱测量（cystometry）会显示她的残余尿量和逼尿肌收缩情况。

首先，我知道在像打喷嚏或咳嗽等活动中发生不自主尿液流失通常与压力性尿失禁有关。压力性尿失禁通常发生在尿道肌肉不足以在压力增大的情况下（比如咳嗽时）防止膀胱漏尿时。

接下来是Q-tip测试。根据我记得的，Q-tip是一种用于测量尿道压力曲线的尿道导管。它通常用于评估尿道功能。Q-tip测试阳性结果，即在Valsalva操作过程中尿道压力低于膀胱内压，与内源性括约肌缺陷相关，这是一种压力性尿失禁类型。

由于患者有在活动中出现不自主漏尿的病史，但夜间没有漏尿，更可能是压力性尿失禁，而不是像急迫性尿失禁那样的情况，急迫性尿失禁通常伴有夜间漏尿。因此，如果Q-tip测试阳性，提示内源性括约肌缺陷。

现在，谈到膀胱测量。膀胱测量是一种测试，旨在测量膀胱在充盈过程中的反应以及逼尿肌的收缩情况。它可以显示是否存在膀胱过度活动症（OAB），即引起急迫感和频尿的情况，或是否存在逼尿肌低活动性，导致尿潴留。

在这种情况下，患者的主要问题是压力性尿失禁，这更与无法在压力增大时保持尿液有关。膀胱测量会查看逼尿肌的收缩情况。如果逼尿肌低活动性，它将不能强有力地收缩以排空膀胱，导致残余尿量。但如果逼尿肌过度活跃，可能会收缩过度，导致急迫感。

鉴于患者有压力性尿失禁的病史和Q-tip测试阳性，提示内源性括约肌缺陷，我认为膀胱测量会显示逼尿肌的收缩是正常的。问题不在于逼尿肌收缩的能力，而是无法密封尿道以保持压力。因此，残余尿量可能是正常的，除非有明显的尿潴留，但关键发现是逼尿肌的收缩是正常的，而不是过度活跃。

等等，但会不会有残余尿量？如果患者排尿后膀胱中残留一些尿液，那就是残余尿量。但如果没有尿潴留的症状，比如膀胱饱胀或排尿困难，那么这种情况的可能性较小。主要问题是在活动中发生的尿失禁，因此逼尿肌收缩是正常的，残余尿量在正常范围内，除非有其他情况。

所以，综合来看，膀胱测量可能会显示逼尿肌的收缩正常，残余尿量正常。问题更多是在括约肌方面，而不是逼尿肌。  
\</think>

根据对患者病史和Q-tip测试结果的分析，膀胱测量最可能显示逼尿肌的收缩正常，残余尿量正常。主要问题似乎是由于内源性括约肌缺陷引起的压力性尿失禁，如Q-tip测试阳性所示。这种情况通常影响尿道括约肌在压力增大时防止漏尿的能力，而不是逼尿肌的收缩能力。因此，逼尿肌的收缩并未过度活跃，残余尿量在正常范围内。

标准答案：

在这种压力性尿失禁的情况下，膀胱测压检查（cystometry）最可能显示**正常的排尿后残余尿量**，因为压力性尿失禁通常不会影响膀胱排空功能。此外，由于压力性尿失禁主要与**身体用力**有关，而不是膀胱过度活动症（OAB），因此在测试过程中**不太可能观察到逼尿肌的非自主收缩**。

In [42]:
inputs2 = tokenizer([prompt_style.format(question_2, "")], return_tensors="pt").to("cuda")


outputs2 = model.generate(
    input_ids=inputs2.input_ids,
    max_new_tokens=1200,
    use_cache=True,
)

response2 = tokenizer.batch_decode(outputs2)

In [43]:
print(response2[0].split("### Response:")[1])


<think>

Okay, let's tackle this question. The patient has sudden chest pain radiating to the neck and left arm. They have a history of hypercholesterolemia and coronary artery disease. Elevated troponin I suggests myocardial damage, so probably a heart attack. Tachycardia could be a response to pain or the heart issue itself.

First, I need to recall the coronary arteries and their territories. The left main coronary artery branches into the left anterior descending (LAD) and the left circumflex (LCX). The right coronary artery (RCA) is the other main one. 

Chest pain location can sometimes correlate with the artery affected. But radiation to the neck and left arm... Wait, the left arm radiation is classic for cardiac issues. The LAD supplies the anterior wall. The LCX goes around the left side, and the RCA supplies the inferior wall. 

Wait, the left circumflex artery (LCX) might supply the lateral wall, and sometimes the left main artery if it's a proximal block. But the RCA often

翻译如下：

\<think>  
好的，我有一位患者，突然出现胸痛，并放射到颈部和左臂。这让我想到了心脏病发作，因为这些症状很经典——心绞痛或心肌梗死。左臂痛、颈部痛，有时还会伴随下颌或背部的疼痛，这些都可能与冠状动脉问题相关。

从病史来看，患者有**高胆固醇血症**（即高胆固醇）和**冠状动脉疾病**，这两个因素都是**动脉粥样硬化**的风险因素，可能导致冠状动脉发生堵塞。**肌钙蛋白I升高**是一个很大的线索，因为肌钙蛋白是心肌受损时释放的心脏酶，通常表明发生了心肌梗死。另外，患者还出现了**心动过速**，即心跳比平常快。在心肌梗死时，心脏可能会加速跳动，以试图通过增加心脏输出量来补偿被阻塞的冠状动脉。

考虑到冠状动脉，**左主冠状动脉**（LMCA）为整个左侧心脏提供血液，包括左心室，而左心室是一个关键的泵血肌肉。如果这里发生堵塞，可能导致更严重的心肌梗死，因为左心室至关重要。**右冠状动脉**为右心室和左心室下壁提供血液，这里的堵塞也是可能的，但**左主冠状动脉**通常与上述症状更相关，尤其是当肌钙蛋白升高时。

所以，将所有因素综合考虑，最可能受累的冠状动脉是**左主冠状动脉**（LMCA）。患者的病史、肌钙蛋白升高以及典型的胸痛放射症状都指向了这一动脉作为罪魁祸首。

\</think>

最可能受累的冠状动脉是**左主冠状动脉（LMCA）**。

**解释：**
- **症状：** 患者突发胸痛并放射至颈部和左臂，以及肌钙蛋白升高，提示急性冠状动脉综合症，可能是心肌梗死。
- **病史：** 高胆固醇血症和冠状动脉疾病病史是动脉粥样硬化的风险因素，可能导致冠状动脉堵塞。
- **心动过速：** 心率增加可能是心脏为补偿心肌血流减少而产生的反应。
- **冠状动脉考虑：** 左主冠状动脉供应左心室，这个肌肉对心脏功能至关重要。与右冠状动脉相比，左主冠状动脉的堵塞会导致更严重且危及生命的心肌梗死，右冠状动脉通常供应的是不那么关键的区域。

因此，症状、肌钙蛋白升高以及患者的病史强烈指向**左主冠状动脉**（LMCA）作为最可能的罪魁祸首。<｜end▁of▁sentence｜>

标准答案：

根据患者表现出的突然胸痛并放射至颈部和左臂，结合其有高胆固醇血症和冠状动脉疾病的病史，肌钙蛋白升高和心动过速，临床症状强烈提示左前降支（LAD）动脉受累。该动脉通常是引发此类症状的罪魁祸首，因为它供应了心脏的大部分区域。放射性疼痛和肌钙蛋白升高的组合表明心肌受损，这使得LAD成为最可能的致病动脉。然而，在没有进一步的诊断检查（如心电图）的情况下，最终的确诊仍需等待确认。

能够看出，在原始状态下，模型能够进行推理并给出回复，但实际上第一个回答过程并不符合医学规范，而第二个问题则直接回答错误。由此可见，在初始状态下，模型对于medical-o1-reasoning-SFT数据集问答效果并不好。

&emsp;&emsp;接下来尝试进行微调，并测试微调后模型问答效果。

### 三、最小可行性实验

&emsp;&emsp;接下来我们尝试进行模型微调，对于当前数据集而言，我们可以带入原始数据集的部分数据进行微调，也可以带入全部数据并遍历多次进行微调。对于大多数的微调实验，我们都可以从最小可行性实验入手进行微调，也就是先尝试带入少量数据进行微调，并观测微调效果。若微调可以顺利执行，并能够获得微调效果，再考虑带入更多的数据进行更大规模微调。

#### 1.数据集准备

&emsp;&emsp;这里我们直接从huggingface上下载medical-o1-reasoning-SFT数据集。

- 设置代理环境

&emsp;&emsp;由于huggingface网络受限，下载数据集前需要先进行网络环境设置。若是AutoDL服务器，则可以按照如下方式开启学术加速，从而顺利连接huggingface并进行数据集下载：

In [15]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

- 下载数据集

&emsp;&emsp;接下来使用datasets进行数据集下载

In [45]:
!pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [10]:
import os
from datasets import load_dataset

再次确认提示词模板：

In [11]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. 
Please answer the following medical question. 

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

然后提取并设置文本生成结束的标记：

In [12]:
EOS_TOKEN = tokenizer.eos_token
tokenizer.eos_token

'<|im_end|>'

然后定义函数，用于对medical-o1-reasoning-SFT数据集进行修改，Complex_CoT列和Response列进行拼接，并加上文本结束标记：

In [13]:
def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250206180316919.png" alt="image-20250206180316919" style="zoom:50%;" />

在最小可行性实验中，我们可以只下载500条数据进行微调即可看出效果：

In [16]:
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT","en", split = "train[0:500]",trust_remote_code=True)

In [17]:
dataset[0]

{'Question': 'A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?',
 'Complex_CoT': "Okay, let's think about this step by step. There's a 61-year-old woman here who's been dealing with involuntary urine leakages whenever she's doing something that ups her abdominal pressure like coughing or sneezing. This sounds a lot like stress urinary incontinence to me. Now, it's interesting that she doesn't have any issues at night; she isn't experiencing leakage while sleeping. This likely means her bladder's ability to hold urine is fine when she isn't under physical stress. Hmm, that's a clue that we're dealing with something related to pressure rather than a bladder muscle problem. \n\nThe fact that she underwent a Q-tip test is intriguing too. This 

然后进行结构化处理：

In [18]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

将数据集整理为如下形式：

In [19]:
dataset["text"][0]

"Below is an instruction that describes a task, paired with an input that provides further context. \nWrite a response that appropriately completes the request. \nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. \nPlease answer the following medical question. \n\n### Question:\nA 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?\n\n### Response:\n<think>\nOkay, let's think about this step by step. There's a 61-year-old woman here who's been dealing with involuntary urine leakages whenever she's doing something that ups her ab

- 数据集保存地址

默认情况下数据集保存在主目录下.cache文件夹中，数据文件格式如下所示：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250311161225502.png" alt="image-20250311161225502" style="zoom:50%;" />

#### 2.开启微调

&emsp;&emsp;然后即可把模型设置为微调模式：

In [20]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,  
    bias="none",  
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  
    loftq_config=None,
)

Unsloth 2025.3.9 patched 64 layers with 64 QKV layers, 64 O layers and 64 MLP layers.


然后导入相关的库：

In [21]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

创建有监督微调对象：

In [63]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

Tokenizing to ["text"] (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

这段代码主要是用 **`SFTTrainer`** 进行 **监督微调（Supervised Fine-Tuning, SFT）**，适用于 `transformers` 和 `Unsloth` 生态中的模型微调：
**1. 导入相关库**
- **`SFTTrainer`**（来自 `trl` 库）：  
  - `trl`（Transformer Reinforcement Learning）是 Hugging Face 旗下的 `trl` 库，提供 **监督微调（SFT）** 和 **强化学习（RLHF）** 相关的功能。
  - `SFTTrainer` 主要用于 **有监督微调（Supervised Fine-Tuning）**，适用于 `LoRA` 等低秩适配微调方式。

- **`TrainingArguments`**（来自 `transformers` 库）：  
  - 这个类用于定义 **训练超参数**，比如批量大小、学习率、优化器、训练步数等。

- **`is_bfloat16_supported()`**（来自 `unsloth`）：  
  - 这个函数检查 **当前 GPU 是否支持 `bfloat16`（BF16）**，如果支持，则返回 `True`，否则返回 `False`。
  - `bfloat16` 是一种更高效的数值格式，在 **新款 NVIDIA A100/H100** 等 GPU 上表现更优。

**2. 初始化 `SFTTrainer` 进行模型微调**

##### **参数解析**
##### **① `SFTTrainer` 部分**
| 参数 | 作用 |
|------|------|
| `model=model` | 指定需要进行微调的 **预训练模型** |
| `tokenizer=tokenizer` | 指定 **分词器**，用于处理文本数据 |
| `train_dataset=dataset` | 传入 **训练数据集** |
| `dataset_text_field="text"` | 指定数据集中哪一列包含 **训练文本**（在 `formatting_prompts_func` 里处理） |
| `max_seq_length=max_seq_length` | **最大序列长度**，控制输入文本的最大 Token 数量 |
| `dataset_num_proc=2` | **数据加载的并行进程数**，提高数据预处理效率 |

##### **② `TrainingArguments` 部分**
| 参数 | 作用 |
|------|------|
| `per_device_train_batch_size=2` | 每个 **GPU/设备** 的训练批量大小（较小值适合大模型） |
| `gradient_accumulation_steps=4` | **梯度累积步数**（相当于 `batch_size=2 × 4 = 8`） |
| `warmup_steps=5` | **预热步数**（初始阶段学习率较低，然后逐步升高） |
| `max_steps=60` | **最大训练步数**（控制训练的总步数，此处总共约消耗60*8=480条数据） |
| `learning_rate=2e-4` | **学习率**（`2e-4` = 0.0002，控制权重更新幅度） |
| `fp16=not is_bfloat16_supported()` | 如果 **GPU 不支持 `bfloat16`，则使用 `fp16`（16位浮点数）** |
| `bf16=is_bfloat16_supported()` | 如果 **GPU 支持 `bfloat16`，则启用 `bfloat16`（训练更稳定）** |
| `logging_steps=10` | **每 10 步记录一次训练日志** |
| `optim="adamw_8bit"` | **使用 `adamw_8bit`（8-bit AdamW优化器）减少显存占用** |
| `weight_decay=0.01` | **权重衰减（L2 正则化）**，防止过拟合 |
| `lr_scheduler_type="linear"` | **学习率调度策略**（线性衰减） |
| `seed=3407` | **随机种子**（保证实验结果可复现） |
| `output_dir="outputs"` | **训练结果的输出目录** |

然后设置wandb（可选）：

In [64]:
import wandb

In [66]:
wandb.login(key="YOUR_WANDB_API_KEY")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 2323365771 (2323365771-ff) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [67]:
run = wandb.init(project='Fine-tune-QwQ-32B-4bit on Medical COT Dataset', )

然后开始微调：

In [68]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 2 | Total steps = 60
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 134,217,728/18,388,423,680 (0.73% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.693200
20,1.248900
30,1.182900
40,1.170200
50,1.144700
60,1.179900


此时wandb中显示内容如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250311162939444.png" alt="image-20250311162939444" style="zoom:33%;" />

In [69]:
trainer_stats

TrainOutput(global_step=60, training_loss=1.2699724833170574, metrics={'train_runtime': 665.1344, 'train_samples_per_second': 1.443, 'train_steps_per_second': 0.09, 'total_flos': 1.740613665982464e+17, 'train_loss': 1.2699724833170574})

注意，unsloth在微调结束后，会自动更新模型权重（在缓存中），因此无需手动合并模型权重即可直接调用微调后的模型：

In [70]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 5120, padding_idx=151654)
        (layers): ModuleList(
          (0): Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=5120, out_features=5120, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=5120, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4

In [71]:
inputs = tokenizer([prompt_style.format(question_1, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=2048,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)

In [72]:
print(response[0].split("### Response:")[1])


<think>
Alright, let's think this through. We've got this woman, 61 years old, and she's been dealing with involuntary urine leakage for a while, especially during activities like coughing or sneezing. That's classic stress urinary incontinence, right? It usually happens when there's some pressure on the bladder from things like laughing or lifting something heavy.

Now, she's been checked out by a gynecologist and they did a Q-tip test. From what I remember, the Q-tip test is all about figuring out how well the urethra is supported. If the angle is too steep, that could mean the urethra isn't supported properly, which is a sign of stress incontinence.

So, if she's got stress urinary incontinence, what does that say about her bladder's behavior? Stress incontinence is mainly about the urethral support and not about the bladder muscle itself. That means the bladder muscle, or the detrusor, isn't really misbehaving here. It should be calm and not causing any issues.

Now, let's talk ab

测试第二个问题问答效果：

In [73]:
inputs = tokenizer([prompt_style.format(question_2, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=2048,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)

In [74]:
print(response[0].split("### Response:")[1])


<think>
Okay, let's think about this. The patient is having chest pain that's suddenly come on, and it's radiating to the neck and left arm. That sounds like classic angina, maybe even a heart attack. The fact that it's radiating is a big clue—it often means it's coming from the heart, not something else.

Now, the patient has a history of hypercholesterolemia and coronary artery disease. So, we know they've got some pre-existing issues with their heart. They're also showing elevated troponin I levels, which is a sign of heart muscle damage. That really makes me think they're having a heart attack, or myocardial infarction.

And then there's the tachycardia. That's a fast heart rate, which could be a reaction to the stress of a heart attack or just the pain itself.

Now, let's think about which coronary artery is likely involved. The left anterior descending artery, or LAD, is really important because it supplies a big chunk of the heart, including the anterior wall and the septum. If

> 此时模型认为“左主冠状动脉最可能是该患者症状的罪魁祸首”，但实际上应该是“左前降支（LAD）动脉受累”导致。

能够发现，第一个问题回答更加规范，并且回答正确。但第二个问题仍然回答错误。由此可以考虑继续进行大规模微调。不过在此之前，我们可以将现在小规模微调的模型进行本地保存。

#### 3.模型合并

此时本地保存的模型权重在`outputs`文件夹中：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250311163146066.png" alt="image-20250311163146066" style="zoom:50%;" />

然后可使用如下代码进行模型权重合并：

In [76]:
model.save_pretrained_merged("QwQ-Medical-COT-Tiny", tokenizer, save_method = "merged_4bit",)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 657.42 out of 1007.51 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 56%|████████████████████████████████████████████████████▎                                        | 36/64 [00:01<00:01, 25.68it/s]
We will save to Disk and not RAM now.
100%|█████████████████████████████████████████████████████████████████████████████████████████████| 64/64 [00:59<00:00,  1.08it/s]


Unsloth: Saving tokenizer... Done.
Done.


In [30]:
# model.save_pretrained_merged("QwQ-Medical-COT-Tiny", tokenizer, save_method = "merged_16bit",)

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250311165148556.png" alt="image-20250311165148556" style="zoom:50%;" />

此外，也可以使用如下代码将其保存为GGUF格式，方便使用ollama进行推理。本部分导出与合并需要较长时间（约20分钟左右），请耐心等待。

In [77]:
model.save_pretrained_gguf("QwQ-Medical-COT-Tiny-GGUF", tokenizer, quantization_method = "q4_k_m")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 744.1 out of 1007.51 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 64/64 [00:43<00:00,  1.46it/s]


Unsloth: Saving tokenizer... Done.
Done.


Unsloth: Converting qwen2 model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at QwQ-Medical-COT-Tiny-GGUF into bf16 GGUF format.
The output location will be /root/autodl-tmp/QwQ-Medical-COT-Tiny-GGUF/unsloth.BF16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: QwQ-Medical-COT-Tiny-GGUF
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00014.safetensors'
INFO:hf-to-gguf:token_embd.weight,         torch.bfloat16 --> BF16, shape 

In [ ]:
#model.save_pretrained_gguf("dir", tokenizer, quantization_method = "q8_0")
#model.save_pretrained_gguf("dir", tokenizer, quantization_method = "f16")

创建完GGUF文件后如下图所示：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250311173422135.png" alt="image-20250311173422135" style="zoom:50%;" />

- 微调后模型推理

&emsp;&emsp;为避免变量名称冲突，这里需要先重启Jupyter Kernel，然后微调模型保存和推理。

In [27]:
from unsloth import FastLanguageModel

In [28]:
max_seq_length = 2048 
dtype = None 
load_in_4bit = True

In [29]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/root/autodl-tmp/QwQ-Medical-COT-Tiny",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.3.9: Fast Qwen2 patching. Transformers: 4.49.0. vLLM: 0.7.3.
   \\   /|    NVIDIA H800 PCIe. Num GPUs = 2. Max memory: 79.205 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

此时model就是读取进来模型就是微调后模型。

In [31]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 5120, padding_idx=151654)
    (layers): ModuleList(
      (0-63): 64 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=True)
          (k_proj): Linear4bit(in_features=5120, out_features=1024, bias=True)
          (v_proj): Linear4bit(in_features=5120, out_features=1024, bias=True)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=27648, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=27648, bias=False)
          (down_proj): Linear4bit(in_features=27648, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((5120,), eps=1e-05)
        (post_attention_layernorm): Qwen2RMSNorm

In [32]:
tokenizer

Qwen2TokenizerFast(name_or_path='/root/autodl-tmp/QwQ-Medical-COT-Tiny', vocab_size=151643, model_max_length=131072, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|vision_pad|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, single_

然后设置为推理模式：

In [34]:
FastLanguageModel.for_inference(model) 

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 5120, padding_idx=151654)
    (layers): ModuleList(
      (0-63): 64 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=True)
          (k_proj): Linear4bit(in_features=5120, out_features=1024, bias=True)
          (v_proj): Linear4bit(in_features=5120, out_features=1024, bias=True)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=27648, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=27648, bias=False)
          (down_proj): Linear4bit(in_features=27648, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((5120,), eps=1e-05)
        (post_attention_layernorm): Qwen2RMSNorm

并设置提示词模板：

In [35]:
prompt_style_chat = """请写出一个恰当的回答来完成当前对话任务。

### Instruction:
你是一名助人为乐的助手。

### Question:
{}

### Response:
<think>{}"""

然后即可开始进行对话：

In [36]:
question = "你好，好久不见！"

In [37]:
[prompt_style_chat.format(question, "")]

['请写出一个恰当的回答来完成当前对话任务。\n\n### Instruction:\n你是一名助人为乐的助手。\n\n### Question:\n你好，好久不见！\n\n### Response:\n<think>']

In [38]:
inputs = tokenizer([prompt_style_chat.format(question, "")], return_tensors="pt").to("cuda")

In [39]:
outputs = model.generate(
    input_ids=inputs.input_ids,
    max_new_tokens=2048,
    use_cache=True,
)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [40]:
response = tokenizer.batch_decode(outputs)

In [41]:
response

['请写出一个恰当的回答来完成当前对话任务。\n\n### Instruction:\n你是一名助人为乐的助手。\n\n### Question:\n你好，好久不见！\n\n### Response:\n<think>\n嗯，用户发来“你好，好久不见！”，我需要回应。首先，应该表达同样的问候，比如“你好啊！”然后，用户提到“好久不见”，说明之前可能有段时间没联系了，应该回应说见到他们很高兴，或者问最近怎么样。比如可以说“很高兴又见到你！最近怎么样？”。这样既回应了对方的问候，又开启了进一步的对话，让用户有机会分享近况。另外，要保持友好和热情的语气，符合助人为乐助手的角色。可能还要注意不要问得太深入，保持自然和轻松。所以综合起来，回复应该是：“你好啊！很高兴又见到你，最近怎么样？有什么我可以帮你的吗？”这样既友好，又提供了帮助的可能，符合用户作为助手的设定。\n</think>\n\n你好啊！很高兴又见到你，最近怎么样？有什么我可以帮你的吗？<|im_end|>']

In [42]:
print(response[0].split("### Response:")[1])


<think>
嗯，用户发来“你好，好久不见！”，我需要回应。首先，应该表达同样的问候，比如“你好啊！”然后，用户提到“好久不见”，说明之前可能有段时间没联系了，应该回应说见到他们很高兴，或者问最近怎么样。比如可以说“很高兴又见到你！最近怎么样？”。这样既回应了对方的问候，又开启了进一步的对话，让用户有机会分享近况。另外，要保持友好和热情的语气，符合助人为乐助手的角色。可能还要注意不要问得太深入，保持自然和轻松。所以综合起来，回复应该是：“你好啊！很高兴又见到你，最近怎么样？有什么我可以帮你的吗？”这样既友好，又提供了帮助的可能，符合用户作为助手的设定。
</think>

你好啊！很高兴又见到你，最近怎么样？有什么我可以帮你的吗？<|im_end|>


此时显存约20G左右：
<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250311191243266.png" alt="image-20250311191243266" style="zoom:50%;" />

- transformers推理

为了避免重复读取占用显存，这里也可以先重启kernel，再执行以下代码：

In [1]:
from modelscope import AutoModelForCausalLM, AutoTokenizer

model_name = "/root/autodl-tmp/QwQ-Medical-COT-Tiny"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "你好，好久不见！"
messages = [
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

嗯，用户跟我说“你好，好久不见！”，看起来挺开心的。我得先回应一下他的热情，毕竟好久没联系了，应该让他感受到我的热情和关心。然后，我得想一想他为什么会突然联系我，是不是遇到了什么问题呢？或者他只是想聊聊近况？这时候我得保持开放的态度，让他知道我很乐意帮忙或者倾听。

接下来，我应该鼓励他多说一些，这样我们才能更好地了解他的需求。比如问他最近过得怎么样，或者有什么想分享的事情。不过，我得注意别太急，让他感觉自然一些。毕竟有时候人们可能不太愿意一开始就透露太多。

另外，我得确保我的回答既友好又专业，不能太随便也不能太生硬。要让他觉得我们之间的交流既轻松又有效。可能的话，可以举几个例子，比如他可能需要帮助解决问题，或者只是想随便聊聊。这样他可以根据自己的情况选择合适的回应。

最后，我得准备好根据他的回应进一步调整对话的方向。如果他提到具体的问题，我需要深入讨论；如果他只是想闲聊，那就保持轻松的氛围。总之，我的目标是让他感到舒适，并且愿意继续交流下去。
</think>

你好！好久不见，确实有些日子没联系了呢。最近过得怎么样？有什么新鲜事或者需要帮忙的吗？我很期待听听你的近况！


能够看出对话正常。

- ollama推理

这里我们需要将创建的Q4_K_M模型权重单独拷贝一份，然后再编写ModelFile文件，便于ollama调用。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250311200625855.png" alt="image-20250311200625855" style="zoom:33%;" />

```bash
cd /root/autodl-tmp/QwQ-Medical-COT-Tiny-GGUF
mkdir ./unsloth_Q4
cp unsloth.Q4_K_M.gguf ./unsloth_Q4
```

然后创建ModelFile：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250311201030660.png" alt="image-20250311201030660" style="zoom:50%;" />

并输入如下内容：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250311201546078.png" alt="image-20250311201546078" style="zoom:50%;" />

```bash
FROM ./unsloth.Q4_K_M.gguf

TEMPLATE """
请写出一个恰当的回答来完成当前对话任务。

### Instruction:
你是一名助人为乐的助手。

### Question:
{{ .Prompt }}

### Response:
<think>{{ .Response }}<|im_end|>
"""

PARAMETER stop "<|im_end|>"
PARAMETER stop "<|end_of_text|>"
PARAMETER stop "<|reserved_special_token_>"
PARAMETER temperature 1.5
PARAMETER min_p 0.1
```

保存并退出，然后进行模型注册：

```bash
ollama create unsloth_model -f /root/autodl-tmp/QwQ-Medical-COT-Tiny-GGUF/unsloth_Q4/ModelFile
```

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250311201143237.png" alt="image-20250311201143237" style="zoom:50%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250311201322617.png" alt="image-20250311201322617" style="zoom:50%;" />

然后即可开始调用了：

In [36]:
from openai import OpenAI
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',  # required but ignored
)
prompt = "你好，好久不见！"
messages = [
    {"role": "user", "content": prompt}
]
response = client.chat.completions.create(
    messages=messages,
    model='unsloth_model',
)

print(response.choices[0].message.content)


嗯，用户跟我说“你好，好久不见！”，听起来他们可能很久没联系我了。这说明我们之间有一段时间没有交流了，也许之前有过一些互动或者认识？这时候我应该先回应他们的问候，表达见到他们很高兴的感觉。

然后，我需要确认一下用户现在的情况如何，是否一切顺利。这样可以让对话更自然地延续下去，同时也能让用户感到被关心和支持。接下来，可以问问用户有什么特别的事情想分享，这样既展示了我对他们的兴趣，也鼓励他们继续交流。

最后，保持友好和亲切的态度很重要，让用户觉得轻松自在，愿意进一步沟通。总之，我需要确保回应既热情又体贴，帮助我们重新建立联系，并开启一个愉快的对话。
</think>
你好！是啊，确实好久不见了呢。最近过得怎么样？有什么特别的事情想分享吗？很高兴再次见到你！


- vLLM推理

相比之下，vLLM的调用过程就简单多了，只需要在后台开启服务即可：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250311201355019.png" alt="image-20250311201355019" style="zoom:50%;" />

```bash
CUDA_VISIBLE_DEVICES=0,1 vllm serve /root/autodl-tmp/QwQ-Medical-COT-Tiny --tensor-parallel-size 2
```

In [1]:
from openai import OpenAI
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

prompt = "你好，好久不见！"
messages = [
    {"role": "user", "content": prompt}
]

response = client.chat.completions.create(
    model="/root/autodl-tmp/QwQ-Medical-COT-Tiny",
    messages=messages,
)

print(response.choices[0].message.content)

好的，我现在要处理用户的信息。用户说：“你好，好久不见！”。首先，我要分析用户此时可能的情绪和需求。

看用户的问候，明显是带着亲切感和久别重逢的感觉。这种情况下，用户可能在表达对过去交流的想念，或者希望展开一段愉快的对话。看起来用户的心情是积极而温暖的，这也意味着我应该保持同样的热情回应。

接下来，我需要用合适的语气来回应。根据用户友好和亲切的问候，我应该以相同的方式回复，这样能营造出融洽的氛围。同时，融入一些轻松的幽默元素，让对话更加生动有趣，这通常能增强互动性和亲切感。

为了确保用户能得到最佳的回应，我应该使用简练而真诚的语言。这样不仅能让回答显得自然，还能让用户感到被理解和关心。同时，适当的幽默可以进一步提升对话的轻松感，让用户感到更加自在。

现在，结合我之前提出的几点思考，来写出回应：“哈哈，好久不见！看来你找到我了。很高兴和你重逢，有什么新鲜事要分享吗？或者你有什么需要帮忙的？” 这个回答亲切且有助于推动对话向前发展，符合用户的需求。

最后，我会再检查一下，确保我的回应确实是积极的，并且能够延续与用户的愉快交流。这样，我们就能够一起享受这场轻松而愉快的对话了。
</think>

哈哈，好久不见！看来你找到我了。很高兴和你重逢，有什么新鲜事要分享吗？或者你有什么需要帮忙的？


### 三、完整高效微调实验

&emsp;&emsp;最后，我们尝试带入全部数据进行高效微调，以提升模型微调效果。

In [18]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. 
Please answer the following medical question. 

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

In [19]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN


def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

此时读取全部数据

In [21]:
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT","en", split = "train",trust_remote_code=True)
dataset = dataset.map(formatting_prompts_func, batched = True,)
dataset["text"][0]

Map:   0%|          | 0/25371 [00:00<?, ? examples/s]

"Below is an instruction that describes a task, paired with an input that provides further context. \nWrite a response that appropriately completes the request. \nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. \nPlease answer the following medical question. \n\n### Question:\nA 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?\n\n### Response:\n<think>\nOkay, let's think about this step by step. There's a 61-year-old woman here who's been dealing with involuntary urine leakages whenever she's doing something that ups her ab

In [22]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,  
    bias="none",  
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  
    loftq_config=None,
)

Unsloth 2025.1.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


这里设置epoch为3，遍历3次数据集：

In [26]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        num_train_epochs = 3,
        warmup_steps=5,
        # max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/25371 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 25,371 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 9,513
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
10,1.285900
20,1.262500
30,1.245400
40,1.284500
50,1.319800
60,1.298900
70,1.298400
80,1.251900
90,1.295300
100,1.244500


这里总共训练约15个小时。

In [29]:
trainer_stats

TrainOutput(global_step=9513, training_loss=1.0824475168592858, metrics={'train_runtime': 20193.217, 'train_samples_per_second': 3.769, 'train_steps_per_second': 0.471, 'total_flos': 2.7936033274397737e+18, 'train_loss': 1.0824475168592858, 'epoch': 2.9992117294655527})

带入两个问题进行测试，均有较好的回答效果：

In [30]:
question = "A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?"


FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
Alright, let's think this through step by step. We've got a 61-year-old woman who's been dealing with involuntary urine loss whenever she does something like coughing or sneezing. That sounds like stress urinary incontinence, which usually means there's some kind of leakage when there's increased pressure in the abdomen. Now, the fact that she doesn't have any leakage at night is interesting. That's a big hint because it suggests that the problem isn't about bladder control or retention, since there's no issue when lying down.

Now, let's consider the Q-tip test. This test is a clever way to check for urethral mobility. If the Q-tip moves a lot, it means the urethra is mobile, which is a typical sign of stress incontinence. So, we're probably looking at a condition where the urethra isn't staying closed under pressure.

Now, if we were to do a cystometry on her, we'd be checking a few things. First, there's the residual volume. Since she's not leaking urine at night, her bladd

In [32]:
question = "Given a patient who experiences sudden-onset chest pain radiating to the neck and left arm, with a past medical history of hypercholesterolemia and coronary artery disease, elevated troponin I levels, and tachycardia, what is the most likely coronary artery involved based on this presentation?"


FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
Okay, let's think about this. The patient is having sudden chest pain that's shooting to their neck and left arm. That's pretty classic for something like a myocardial infarction, right? I mean, the pain spreading like that usually points to a big problem in the heart's supply.

Now, considering the heart's coronary arteries, I know the left anterior descending artery, or LAD, is a big player. It's like the main highway for blood to reach the front wall of the heart. If there's a blockage there, it can definitely cause pain that radiates to the neck and arm.

Then, there's the right coronary artery, or RCA, which supplies the right side of the heart and can affect the inferior wall of the heart. But wait, the pain pattern here seems to be more on the left side, so maybe the RCA is less likely.

The patient has hypercholesterolemia and coronary artery disease. These conditions put them at risk for atherosclerosis, which can lead to blockages in the coronary arteries. The LAD is